### Youtube Sentence Adjust Word List Ratio

In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import glob

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# parameter
folder_name = "Turkish"
word_num = 28
adjust_sent_word_ratio = 100
adjust_videoid_text_word_ratio = 100
sample_sent_num = 6

#### Calculate Sentence Ratio By Word

In [4]:
# original part
#df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
#df_adjust_word = df_word.iloc[0:word_num,]

# variable part
#df_ety_prefix = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish Intersect/Turkish_Intersect_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
df_ety_prefix = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish English/Turkish_English_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
df_adjust_word = df_ety_prefix

df_adjust_word

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
2130,şok,şokta
2131,şok,şoku
2132,şırınga,şırınga
2133,şırınga,şırıngayla


In [5]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [6]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [7]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"özgür, büyük, kullanılıyor, kamyon, araçlar","için, bunlar, daha, normalde"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"tarafı, burcu, kolay, arka, olmayacak, bağlamak",
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"özgür, attıracağım, arabaya, yarım, tur",
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,22.3,"tamam, devam, burcu, oldu, et","daha, biraz"
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"burcu, artarsa, şimdilik, takacağız, kalınlığı...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,33.4,"yükselmez, herkes, umarız, güçlü, ellerde, büy...","teknoloji, bu, ve, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"herkesin, faydalı, halde, yararlanacağı, geliş...","bir, daha, ve"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"olarak, link, belirttim, yararlandığım, kaynak...",


In [8]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"özgür, büyük, kullanılıyor, kamyon, araçlar","için, bunlar, daha, normalde"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"tarafı, burcu, kolay, arka, olmayacak, bağlamak",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"özgür, attıracağım, arabaya, yarım, tur",NaN
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,22.3,"tamam, devam, burcu, oldu, et","daha, biraz"
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"burcu, artarsa, şimdilik, takacağız, kalınlığı...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,33.4,"yükselmez, herkes, umarız, güçlü, ellerde, büy...","teknoloji, bu, ve, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"herkesin, faydalı, halde, yararlanacağı, geliş...","bir, daha, ve"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"olarak, link, belirttim, yararlandığım, kaynak...",NaN


In [9]:
df_sentence_ratio_result.to_csv(f"Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv", index=False)

#### Select Adjust Sentence Ratio Result

In [10]:
#df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"özgür, büyük, kullanılıyor, kamyon, araçlar","için, bunlar, daha, normalde"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"tarafı, burcu, kolay, arka, olmayacak, bağlamak",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"özgür, attıracağım, arabaya, yarım, tur",NaN
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,22.3,"tamam, devam, burcu, oldu, et","daha, biraz"
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"burcu, artarsa, şimdilik, takacağız, kalınlığı...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,33.4,"yükselmez, herkes, umarız, güçlü, ellerde, büy...","teknoloji, bu, ve, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"herkesin, faydalı, halde, yararlanacağı, geliş...","bir, daha, ve"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"olarak, link, belirttim, yararlandığım, kaynak...",NaN


In [14]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result[f"{word_num}_word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,222,00:27:46.048,00:27:46.560,evet,8V9tq1pe8eI,100.1,NaN,evet
1,510,00:04:48.400,00:04:49.280,o da,cQjX9DzXTw0,100.1,NaN,"o, da"
2,605,00:10:35.360,00:10:36.520,evet,cQjX9DzXTw0,100.1,NaN,evet
3,789,00:07:25.787,00:07:27.755,genelde,NRa1iXv3OLw,100.1,NaN,genelde
4,894,00:20:16.397,00:20:19.366,model,NRa1iXv3OLw,100.1,NaN,model
...,...,...,...,...,...,...,...,...
47098,3036709,00:17:25.970,00:17:26.000,bu,sSTIfQUm18Q,100.1,NaN,bu
47099,3036732,00:20:21.720,00:20:21.780,ben,sSTIfQUm18Q,100.1,NaN,ben
47100,3036760,00:25:30.370,00:25:30.400,bu,sSTIfQUm18Q,100.1,NaN,bu
47101,3036762,00:25:54.640,00:25:54.700,sen,sSTIfQUm18Q,100.1,NaN,sen


In [15]:
df_adjust_sentence_ratio.sort_values("sentence",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
16757,1107468,00:17:56.080,00:17:57.780,senin için çok normal bir şey değil mi bu,kSIzlvs8ngw,100.1,NaN,"bu, mi, değil, çok, normal, bir, şey, senin, için"
36408,2225948,01:23:00.940,01:23:03.100,hayır senin gibi diplomam da yok benim,e7RXMoofC1A,100.1,NaN,"gibi, senin, da, diplomam, yok, benim, hayır"
45628,2974710,00:01:59.167,00:02:02.100,burada da tiyatro ve restoran varmış,SrpqpSKIdQw,100.1,NaN,"tiyatro, burada, varmış, restoran, da, ve"
16701,1104855,00:08:55.510,00:08:58.165,ben alternatif bir model değilim ama,MBSMO8akXno,100.1,NaN,"değilim, model, alternatif, bir, ben, ama"
20290,1308956,00:58:21.180,00:58:24.020,benim onunla bir problemim yok ama,L2u31Yt7r2w,100.1,NaN,"onunla, bir, problemim, yok, benim, ama"
...,...,...,...,...,...,...,...,...
9474,673595,00:00:44.448,00:00:45.039,o,XNt_vFwAcFI,100.1,NaN,o
42087,2637320,00:05:35.780,00:05:37.200,o,crxe2d7kvvg,100.1,NaN,o
4673,387897,00:09:01.840,00:09:02.840,o,lCBYTVAQOwU,100.1,NaN,o
42101,2639279,00:00:00.000,00:00:06.810,o,6dEyHaXpGaM,100.1,NaN,o


In [ ]:
#df_adjust_sentence_ratio.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv", index=False)

In [16]:
df_adjust_sentence_ratio.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.xlsx", index=False)

#### Calculate Videoid Text Ratio By Word

In [29]:
# original part
#df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
#df_adjust_word = df_word.iloc[0:word_num,]

# variable part
#df_ety_prefix = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish Intersect/Turkish_Intersect_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
df_ety_prefix = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish English/Turkish_English_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
df_adjust_word = df_ety_prefix

df_adjust_word

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
6355,şut,şutu
6356,şut,şutunu
6357,şırınga,şırınga
6358,şırınga,şırıngayla


In [18]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [19]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

,index,video_id,sentence
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...,...
17990,17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
17991,17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
17992,17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
17993,17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [20]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [21]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence",f"{word_num}_word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,8.4,"bıraktım, kuymak, kaşar, yerine, gördüğünüz, a...","var, bu, onun, gibi, çok, bugün, kadar, de, bu..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,3.5,"yıllık, işte, anlatamayız, yerine, insanlarımı...","finans, demokrasiye, burada, onu, bunlar, pilo..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"birbirinizi, h, l, hasretimsini, idağhan, iyok...","mi, ne, biraz, de, ben, sen, benim"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"nazlı, birbirinden, farkında, yalnız, herkes, ...","onu, sen, da, benim, var, bu, beni, gibi, deği..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.2,"perspective, children, george, symptoms, need,...",risk
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,5.2,"bembeyaz, sağlarım, avrupa, dayanıklıyımdır, h...","parfüm, sistemini, seni, bana, vitamini, varsa..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"nazlı, hanım, başla, öldürürüm, hiçkimse, elim...","bu, mi, gibi, birisi, bana, seni, yoksa, de, v..."
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,3.0,"odamızda, buçuk, durum, istiyorum, karşıdaki, ...","sistemi, bununla, atmosfer, burada, onu, buran..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,21.0,"film, yaptığıma, taşınabilirliğe, bakabileceği...","çok, kadar, vardı, ben, bir, ve, bunun, için, ..."


In [22]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,8.4,"bıraktım, kuymak, kaşar, yerine, gördüğünüz, a...","var, bu, onun, gibi, çok, bugün, kadar, de, bu..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,3.5,"yıllık, işte, anlatamayız, yerine, insanlarımı...","finans, demokrasiye, burada, onu, bunlar, pilo..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"birbirinizi, h, l, hasretimsini, idağhan, iyok...","mi, ne, biraz, de, ben, sen, benim"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"nazlı, birbirinden, farkında, yalnız, herkes, ...","onu, sen, da, benim, var, bu, beni, gibi, deği..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.2,"perspective, children, george, symptoms, need,...",risk
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,5.2,"bembeyaz, sağlarım, avrupa, dayanıklıyımdır, h...","parfüm, sistemini, seni, bana, vitamini, varsa..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"nazlı, hanım, başla, öldürürüm, hiçkimse, elim...","bu, mi, gibi, birisi, bana, seni, yoksa, de, v..."
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,3.0,"odamızda, buçuk, durum, istiyorum, karşıdaki, ...","sistemi, bununla, atmosfer, burada, onu, buran..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,21.0,"film, yaptığıma, taşınabilirliğe, bakabileceği...","çok, kadar, vardı, ben, bir, ve, bunun, için, ..."


In [23]:
df_text_ratio_result.to_csv(f"Youtube_Videoid_Text_{word_num}_Word_All_Ratio_Result.csv", index=False)

In [ ]:
df_text_ratio_result.to_excel(f"Youtube_Videoid_Text_{word_num}_Word_All_Ratio_Result.xlsx", index=False)

In [ ]:
# 2 Way
#df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
#df_videoid_sentence

In [ ]:
#def videoid_word_ratio(df_word, df_videoid_sentence, word_column_name, videoid_sentence_column_name, word_num):
#    '''
#    videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", word_num)
#    '''
#    word_set = set(df_word[f"{word_column_name}"].to_list())
#    for i in range(len(df_videoid_sentence)):
#        text = df_videoid_sentence.loc[i,f"{videoid_sentence_column_name}"]
#        text_word = re.findall(r"\w+", text, re.UNICODE)
#        text_word_set = set(text_word)
#        intersect_word = list(word_set.intersection(text_word_set))
#        different_word = list(text_word_set.difference(word_set))
#        df_videoid_sentence.loc[i,f"{word_num}_word_ratio"] = round(((len(intersect_word)/len(text_word)+0.001)*100),)
#        df_videoid_sentence.loc[i,"different_word"] = ", ".join(different_word)
#        df_videoid_sentence.loc[i,"intersect_word"] = ", ".join(intersect_word)
#        
#    return df_videoid_sentence

In [ ]:
#df_text_ratio_result = videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", word_num)
#df_text_ratio_result

In [ ]:
#df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
#df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
#df_text_ratio_result 

In [ ]:
#df_text_ratio_result.to_csv(f"Youtube_Videoid_Text_{word_num}_Word_All_Ratio_Result.csv", index=False)

#### Select Adjust Videoid Text Ratio Result

In [24]:
#df_text_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Videoid_Text_{word_num}_Word_All_Ratio_Result.csv")
df_text_ratio_result = pd.read_csv(f"Youtube_Videoid_Text_{word_num}_Word_All_Ratio_Result.csv")
df_text_ratio_result

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,8.4,"bıraktım, kuymak, kaşar, yerine, gördüğünüz, a...","var, bu, onun, gibi, çok, bugün, kadar, de, bu..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,3.5,"yıllık, işte, anlatamayız, yerine, insanlarımı...","finans, demokrasiye, burada, onu, bunlar, pilo..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"birbirinizi, h, l, hasretimsini, idağhan, iyok...","mi, ne, biraz, de, ben, sen, benim"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"nazlı, birbirinden, farkında, yalnız, herkes, ...","onu, sen, da, benim, var, bu, beni, gibi, deği..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.2,"perspective, children, george, symptoms, need,...",risk
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,5.2,"bembeyaz, sağlarım, avrupa, dayanıklıyımdır, h...","parfüm, sistemini, seni, bana, vitamini, varsa..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"nazlı, hanım, başla, öldürürüm, hiçkimse, elim...","bu, mi, gibi, birisi, bana, seni, yoksa, de, v..."
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,3.0,"odamızda, buçuk, durum, istiyorum, karşıdaki, ...","sistemi, bununla, atmosfer, burada, onu, buran..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,21.0,"film, yaptığıma, taşınabilirliğe, bakabileceği...","çok, kadar, vardı, ben, bir, ve, bunun, için, ..."


In [25]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result[f"{word_num}_word_ratio"] >= adjust_videoid_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,0gPC8dJJqW8,müzik,100.1,NaN,müzik
1,2LqpcYH8BVk,bu,100.1,NaN,bu
2,3ceSnmV4vPc,ben burada,100.1,NaN,"ben, burada"
3,But0WoHNolU,burda,100.1,NaN,burda
4,DY9uKXE8Gp4,müzik,100.1,NaN,müzik
5,PnwSr5f9w1U,müzik,100.1,NaN,müzik
6,RE5IgCDyujE,müzik,100.1,NaN,müzik
7,Req5saUBkZw,müzik,100.1,NaN,müzik
8,TosArq0B6S0,müzik,100.1,NaN,müzik
9,_fwSPHnO4eM,müzik,100.1,NaN,müzik


In [26]:
df_adjust_text_ratio.sort_values("sentence",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_adjust_text_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,video_id,sentence,28_word_ratio,different_word,intersect_word
2,3ceSnmV4vPc,ben burada,100.1,NaN,"ben, burada"
0,0gPC8dJJqW8,müzik,100.1,NaN,müzik
3,But0WoHNolU,burda,100.1,NaN,burda
4,DY9uKXE8Gp4,müzik,100.1,NaN,müzik
5,PnwSr5f9w1U,müzik,100.1,NaN,müzik
6,RE5IgCDyujE,müzik,100.1,NaN,müzik
7,Req5saUBkZw,müzik,100.1,NaN,müzik
8,TosArq0B6S0,müzik,100.1,NaN,müzik
9,_fwSPHnO4eM,müzik,100.1,NaN,müzik
10,pM74dDjMt4s,müzik,100.1,NaN,müzik


In [ ]:
#df_adjust_text_ratio.to_csv(f"Youtube_Videoid_{word_num}_Word_{adjust_videoid_text_word_ratio}_Ratio_Text.csv", index=False)

In [27]:
df_adjust_text_ratio.to_excel(f"Youtube_Videoid_{word_num}_Word_{adjust_videoid_text_word_ratio}_Ratio_Text.xlsx", index=False)

In [28]:
df_text_ratio_result.sort_values(by="28_word_ratio", ascending=False)

,video_id,sentence,28_word_ratio,different_word,intersect_word
191,0gPC8dJJqW8,müzik,100.1,NaN,müzik
3382,But0WoHNolU,burda,100.1,NaN,burda
7816,RE5IgCDyujE,müzik,100.1,NaN,müzik
7401,PnwSr5f9w1U,müzik,100.1,NaN,müzik
655,2LqpcYH8BVk,bu,100.1,NaN,bu
...,...,...,...,...,...
11867,e_9WHq2UhQ8,yavrulayan menekşeleri n anne yapraklari nasil...,0.1,"menekşe, nasil, afrikanviolet, yapraklari, ann...",NaN
16806,vmp_akmR2z0,iyi seyirer,0.1,"seyirer, iyi",NaN
13417,k6ub3xNxE_I,kirmizi kirmizi beyaz beyaz kirmizi kirmizi be...,0.1,"beyaz, ye, abone, ol, kirmizi, en, büyük, türki",NaN
10461,_U_EyQlPbiE,şimdi tut nefesi ver nefes ellerin gelsin kalb...,0.1,"düşünceler, mücadele, ver, tut, gelsin, onlarl...",NaN


#### Calculate Sequential Sentence Length Of Adjust Ratio Result

In [ ]:
#df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result

In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].astype("object")
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].astype("object")

In [ ]:
#df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

In [ ]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [ ]:
df_adjust_ratio["with_word"] = 1
df_adjust_ratio

In [ ]:
df_adjust_ratio_flag = pd.merge(df_sentence_ratio_result , df_adjust_ratio, how="left", on=["index","sentence","start_time","end_time","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_adjust_ratio_flag["with_word"].fillna(0, inplace=True)
df_adjust_ratio_flag

In [ ]:
df_adjust_ratio_flag.with_word.value_counts()

In [ ]:
# Code Running For Video_id Sentence
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_adjust_ratio_flag)):
    videoid = df_adjust_ratio_flag.loc[i,"video_id"]    
    if df_adjust_ratio_flag.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    try:
        if df_adjust_ratio_flag.loc[i+1,"video_id"] != videoid:
            try:
                one_list.append((ones[0],len(ones)))
                ones = []
            except:
                pass
            try:
                zero_list.append((zeros[0],len(zeros)))
                zeros = []
            except:
                pass
        else:
            pass
    except:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [ ]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one.sort_values(by="length", ascending=False, inplace=True)
df_one.reset_index(drop=True, inplace=True)
df_one

In [ ]:
df_one_all = pd.DataFrame()
for i in range(len(df_one)):
    index = df_one.loc[i,"index"]
    length = df_one.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_all = pd.concat([df_one_all, df_video],axis=0)
df_one_all.reset_index(inplace=True, drop=True)
df_one_all

In [ ]:
#df_one_all.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentences_And_Others.csv", index=False)

In [ ]:
# Only Sequential Sentences
df_one_sequential = df_one[df_one.length > 1]
df_one_sequential

In [ ]:
df_one_sequential_all = pd.DataFrame()
for i in range(len(df_one_sequential)):
    index = df_one_sequential.loc[i,"index"]
    length = df_one_sequential.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_sequential_all = pd.concat([df_one_sequential_all, df_video],axis=0)
df_one_sequential_all.reset_index(inplace=True, drop=True)
df_one_sequential_all

In [ ]:
#df_one_sequential_all.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.csv", index=False)

In [ ]:
#df_one_sequential_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.xlsx", index=False)

#### Count Sentence Words By Adjust Column

In [ ]:
#df_one_sequential_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.csv")
df_one_sequential_all = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.csv")
df_one_sequential_all

In [ ]:
def text_to_list_df(df_target, target_column): 
    '''
    df_target include dict value columns, target_column occurs dict \n
    text_to_list_df(df_one_sequential_all, "intersect_word")
    '''
    text_value_list = []
    for i in df_target[f"{target_column}"]:
        try:
            var = re.findall(r"\w+", i)
            for j in var:
                text_value_list.append(j)
        except:
            pass
    df_alone_value = pd.DataFrame(text_value_list)
    df_alone_value.rename(columns={0:f"{target_column}_alone"}, inplace=True)
    df_unique_count = pd.DataFrame(df_alone_value.value_counts(ascending=False))
    df_unique_count.reset_index(inplace=True)
    df_unique_count.rename(columns={0:"count"}, inplace=True)
            
    return df_unique_count

In [ ]:
df_unique_word_count = text_to_list_df(df_one_sequential_all, "intersect_word")
df_unique_word_count

In [ ]:
#df_unique_word_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentence_Words_Count.xlsx", index=False)

#### Count Adjust Sentence Ratio Result By Videoid

In [ ]:
#df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

In [ ]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [ ]:
df_ratio_count = pd.DataFrame(df_adjust_ratio.groupby("video_id")[f"{word_num}_word_ratio"].count())
df_ratio_count.reset_index(inplace=True)
df_ratio_count.rename(columns={f"{word_num}_word_ratio":f"{word_num}_word_ratio_count"}, inplace=True)
df_ratio_count.sort_values(by=f"{word_num}_word_ratio_count", ascending=False, inplace=True)
df_ratio_count.reset_index(drop=True,inplace=True)
df_ratio_count

In [ ]:
#df_ratio_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Videoid_Count.xlsx", index=False)

#### Find Word In Sentence Time Interval

In [ ]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
word_list = df_adjust_word.iloc[:,0].values.tolist()

In [ ]:
df_word_result = pd.DataFrame()
for i in word_list:
    try:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].sample(sample_sent_num)
    except:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(sample_sent_num) 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","start_time"], ascending=True)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

In [ ]:
word_time_loc_list = []
def word_time_loc(df):
    for i in range(len(df)):
        word = df.loc[i,"word"]
        start_time = df.loc[i,"start_time"]
        end_time = df.loc[i,"end_time"]
        sentence = df.loc[i,"sentence"]
        video_id = df.loc[i,"video_id"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=["word","start_time","end_time","sentence","video_id"])

    return df_word_time_loc

In [ ]:
df_word_time_loc_result = word_time_loc(df_word_result)
df_word_time_loc_result

In [ ]:
time_shift = 0.6
df_word_time_loc_result.start_time = df_word_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_time_loc_result.end_time = df_word_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_time_loc_result

In [ ]:
df_word_time_loc_result.start_time = df_word_time_loc_result.start_time.apply(lambda x: round(x))
df_word_time_loc_result.end_time = df_word_time_loc_result.end_time.apply(lambda x: round(x))
df_word_time_loc_result 

In [ ]:
#df_word_time_loc_result.to_excel(f"Youtube_Sentence_{word_num}_Word_Time_Interval.xlsx", index=False)